# Decision Trees

In [7]:
import numpy as np 
import graphviz


class Node:
    def __init__(self,feature=None,value=None,rightchild=None,leftchild=None,leaflabel=None):
        self.feature=feature
        self.value=value
        self.rightchild=rightchild
        self.leftchild=leftchild
        self.leaflabel=leaflabel
    
    def __repr__(self):
        return (f"Node(feature={self.feature},value={self.value}, label={self.leaflabel})")

def CalculateEntropy(labels):
    OccurenceCounts = np.bincount(labels)
    probs = OccurenceCounts/len(labels)
    
    entropy = -np.sum(
        [p * np.log2(p) for p in probs if p > 0]
        )
    return entropy

def split_data(data,labels,feature,value):
    '''
        data = the original data to split 
        labels = the label of the data 
        feature = the features of the data or called as attributes 
        value = the value to compare with the features 
        
        we already know that each node has a (feature, value) pair
    '''
    right_indices = np.where(data[: , feature] > value)[0]
    left_indices = np.where(data[: , feature] <= value)[0]
    
    right_data= data[right_indices]
    left_data= data[left_indices]
    right_labels = labels[right_indices]
    left_labels = labels[left_indices]
    
    return (right_data,right_labels,left_data,left_labels)
    
def buildTree(X,y):
    '''
        X : data set -> nd.array
        y : labels corresponding to the data set -> 1D array 
    '''
    
    # base case -> leaf node 
    if(len(set(y)) == 1):
        return Node(leaflabel=y[0])
    else:
        
        #internal node
        bestfeature=None
        bestValue=None
        bestSplit=None
        bestGain = -np.inf 
        
        CurrentEntropy = CalculateEntropy(y) #of the data-set 
        n_features= X.shape[1]
        
        for feature in range(n_features):
            Values = np.unique(X[:, feature])
            for value in Values:
                right_data, right_labels, left_data, left_labels = split_data(X,y,feature,value)
                if len(right_labels)==0 and len(left_labels)==0:
                    continue
                
                entropy_right = CalculateEntropy(right_labels)
                entropy_left= CalculateEntropy(left_labels)
                
                Information_gain  = CurrentEntropy - (
                        entropy_right * (len(right_labels)/len(y))
                        +
                        entropy_left  * (len(left_labels)/len(y))
                    )
                
                if(Information_gain > bestGain):
                    bestGain = Information_gain
                    bestfeature = feature
                    bestValue = value
                    bestSplit = (right_data, right_labels, left_data, left_labels)
        
        # if no best split was found ,then return the leaf node with class = majority class 
        if bestSplit == None :
            majority_label = np.bincount(y).argmax()
            node = Node(leaflabel=majority_label)
            return node 
        
        # add the node to the tree
        right_data, right_labels, left_data, left_labels = bestSplit
        
        rightchild = buildTree(right_data,right_labels)
        leftchild = buildTree(left_data,left_labels)
        
        root = Node(bestfeature,bestValue,rightchild,leftchild)
        return root     

def predict(tree , sample):
    root = tree 
    
    while True:
        if (root.rightchild is not None) or (root.leftchild is not None):
            attribute = root.feature
            if sample[attribute] >= root.value : 
                root = root.rightchild
            else :
                root = root.leftchild
        else :
            break 
    
    if root.rightchild == None and root.leftchild == None :
        return root.leaflabel
        

def visualize_tree(node , dot=None):
    """
    Recursively creates a DOT representation of the decision tree.
    """
    if dot is None :
        dot = graphviz.Digraph()

    # Create a unique node id for the current node
    node_id = str(id(node))
    
    # If the node is a leaf, just add the label
    if node.leaflabel is not None:
        dot.node(node_id, label=str(node.leaflabel), shape='ellipse', color='blue')
    else:
        # Add internal node with feature and value
        dot.node(node_id, label=f"X{node.feature} <= {node.value:.2f}", shape='box')
        
        if node.leftchild:
            left_id = str(id(node.leftchild))
            dot.node(left_id, label=str(node.leftchild), shape='ellipse')
            dot.edge(node_id, left_id, label="True")
            visualize_tree(node.leftchild, dot)
        
        if node.rightchild:
            right_id = str(id(node.rightchild))
            dot.node(right_id, label=str(node.rightchild), shape='ellipse')
            dot.edge(node_id, right_id, label="False")
            visualize_tree(node.rightchild, dot)

    return dot
                
if __name__ == '__main__':
    X = np.array([[2, 3, 1, 5],
                  [3, 2, 2, 1],
                  [1, 4, 3, 2],
                  [4, 1, 1, 3],
                  [5, 6, 2, 1],
                  [7, 8, 3, 4],
                  [6, 5, 1, 6],
                  [8, 7, 2, 3],
                  [2, 5, 3, 4],
                  [3, 6, 1, 2]])

    y = np.array([0, 0, 0, 1, 1, 1, 0, 1, 0, 1])  # Class labels for each instance
    
    tree = buildTree(X, y)
    print(tree)
    
    sample= np.array([1, 1])
    print(predict(tree , sample))
    
    dot = visualize_tree(tree)
    dot.render("decision_tree", format="png", view=True)  # This will create a PNG image

Node(feature=1,value=5, label=None)
0


# General Binary TreeeI